# Ch5. Network data and network errors

## 5.1 Bytes and strings

In [1]:
# 98 represented in different numerical systems
98 == 0b1100010 == 0o142 == 0x62

True

In [5]:
#  a list of numbers -> bytes
b = bytes([0,11,22,33,99,111,222,255])
b, len(b), type(b), chr(33),chr(99),chr(111)

(b'\x00\x0b\x16!co\xde\xff', 8, bytes, '!', 'c', 'o')

In [6]:
list(b)

[0, 11, 22, 33, 99, 111, 222, 255]

## 5.2 Character strings

In [8]:
for i in range(32,128,32):
    print(' '.join(chr(j) for j in range(i, i+32)))

  ! " # $ % & ' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ?
@ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \ ] ^ _
` a b c d e f g h i j k l m n o p q r s t u v w x y z { | } ~ 


- Encoding: string -> bytes
- Decoding: bytes -> string
- Further info: [codecs — Codec registry and base classes](https://docs.python.org/3/library/codecs.html)

In [10]:
mybytes = '和平万岁！'.encode('utf-8')
mybytes

b'\xe5\x92\x8c\xe5\xb9\xb3\xe4\xb8\x87\xe5\xb2\x81\xef\xbc\x81'

In [11]:
mybytes.decode('latin1')

'å\x92\x8cå¹³ä¸\x87å²\x81ï¼\x81'

In [12]:
mybytes.decode('latin2')

'ĺ\x92\x8cĺšłä¸\x87ĺ˛\x81ďź\x81'

In [13]:
mybytes.decode('greek')

'ε\x92\x8cεΉ³δΈ\x87ε²\x81οΌ\x81'

In [14]:
mybytes.decode('hebrew')

'ו\x92\x8cו¹³ה¸\x87ו²\x81ן¼\x81'

In [15]:
mybytes.decode('utf-8')

'和平万岁！'

In [16]:
'和平万岁！'.encode('utf-16')

b'\xff\xfe\x8cTs^\x07N\x81\\\x01\xff'

In [17]:
'和平万岁！'.encode('utf-32')

b'\xff\xfe\x00\x00\x8cT\x00\x00s^\x00\x00\x07N\x00\x00\x81\\\x00\x00\x01\xff\x00\x00'

In [18]:
mybytes.decode('ascii')

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe5 in position 0: ordinal not in range(128)

In [19]:
'ΛλΘθΩω'.encode('latin-1')

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 0-5: ordinal not in range(256)

In [20]:
mybytes.decode('ascii', 'replace')

'���������������'

In [21]:
mybytes.decode('ascii', 'ignore')

''

In [22]:
'ΛλΘθΩω'.encode('latin-1', 'replace')

b'??????'

In [23]:
'ΛλΘθΩω'.encode('latin-1', 'ignore')

b''

## 5.3 Binary numbers and network byte order

In [28]:
import struct
struct.pack('<i', 0x12abcdef) # little endian

b'\xef\xcd\xab\x12'

In [29]:
struct.pack('>i',0x12abcdef) # big endian

b'\x12\xab\xcd\xef'

In [34]:
struct.unpack('>i',b'\x12\xab\xcd\xef')

(313249263,)

In [35]:
hex(313249263)

'0x12abcdef'

In [36]:
struct.pack('!i',0x12abcdef) # network order

b'\x12\xab\xcd\xef'

## 5.4 How to determine a complete message in the received data?
- **M1**: In TCP blocking mode, recv() returns an empty string indicates the end of a message
  - [Simply Send All Data and Then Close the Connection](./streamer.py)
    ```bash
    # open two terminals, one runs as the server
    python3 streamer.py
    # the other runs the client
    python3 streamer.py -c
    ```
- **M2**: Stream in both directions alternatively
- **M3**: Use fixed-length messages
  ```python
  def recvall(sock, length):
      data = b''
      while len(data) < length:
          more = sock.recv(length - len(data))
          if not more:
              raise EOFError('socket closed {len(data)} bytes into a {length}-byte message.')
          data += more
      return data
  ```
- **M4**: Delimit messages with special characters
  - Hard to choose the delimiter if any data is legal in the message
- **M5**: Prefix each message with its length
  - The length of each message must be known ahead
  - [Framing Each Block of Data by Preceding It with Its Length](./blocks.py)
    ```bash
    # open two terminals, one runs as the server
    python3 blocks.py
    # the other runs the client
    python3 blocks.py -c
    ```
- **M6**: 
  - If the length of message is unknow ahead in **M5**, use it for the current data and
  - Signal the end with length 0

- HTTP uses both **M4** and **M5**
  - the blank line '\r\n\r\n' delimits its header
  - the HTTP payload such as an image is pure binary data,
    - Content-Length is provided in the header to determine the amount of data

## 5.5 Pickles and self-delimiting formats
- pickles have built-in delimiting

In [37]:
import pickle

In [41]:
pa = pickle.dumps([10,'hi',20]) # the period . at the end of the output is the dilimiter
pa

b'\x80\x04\x95\x0e\x00\x00\x00\x00\x00\x00\x00]\x94(K\n\x8c\x02hi\x94K\x14e.'

In [42]:
pickle.loads(pa)

[10, 'hi', 20]

- pickle.load() reads from a file and stops at the end of the pickle data

In [45]:
from io import BytesIO
f = BytesIO(b'\x80\x04\x95\x0e\x00\x00\x00\x00\x00\x00\x00]\x94(K\n\x8c\x02hi\x94K\x14e.more data to come')

In [46]:
pickle.load(f)

[10, 'hi', 20]

In [47]:
f.tell()

25

In [48]:
f.read()

b'more data to come'

- wrap a socket in a Python file object with makefile() then supply to pickle.load()
- [furthe info: pickle — Python object serialization](https://docs.python.org/3/library/pickle.html)

## 5.6 XML and JSON
- widely used data formats
  - no framing support
- JSON is used to exchange data between different programming languages
  - allows Unicode characters in its strings and payload
  - encodes JSON strings as UTF-8 for network transmission
- XML is better for documents

In [50]:
import json
json.dumps([1979, '中美建交'])

'[1979, "\\u4e2d\\u7f8e\\u5efa\\u4ea4"]'

In [51]:
json.dumps([51, 'Nixon and 周恩来'], ensure_ascii=False)

'[51, "Nixon and 周恩来"]'

In [53]:
json.loads('{"USA":"Nixson", "中国":"周恩来"}')

{'USA': 'Nixson', '中国': '周恩来'}

- XML and JSON are text formats
- Binary formats like Thrift and Google Protocol Buffers are more efficient

## 5.7 Compression
- Network throughput is a bottleneck in distributed applications
  - it is worthwhile to compress data before transmission
- GNU zlib a popular compression format
  - available in Python Standard Library (PSL)
  - supports self-framing
  - however, most protocols choose to do their own framing

In [54]:
# there is overhead of compression
# here, two compression streams are separated with '|'
import zlib
data  = zlib.compress(b'First sentence') + b'|' + zlib.compress(b'Second sentence') + b'|'
data

b'x\x9cs\xcb,*.Q(N\xcd+I\xcdKN\x05\x00(d\x05~|x\x9c\x0bNM\xce\xcfKQ(N\xcd+I\xcdKN\x05\x00-\xab\x05\xd2|'

In [55]:
len(data)

47

In [67]:
# suppose network block is of size 8 bytes
d = zlib.decompressobj()
d.decompress(data[0:8]), d.unused_data # empty unused_data indicates more data to come

(b'First', b'')

In [68]:
d.decompress(data[8:16]), d.unused_data

(b' sentenc', b'')

In [69]:
d.decompress(data[16:24]), d.unused_data 
# in '|x', the character followed | belongs to the next compression stream, feed it to a new decompressionobj

(b'e', b'|x')

In [70]:
d2 = zlib.decompressobj()
d2.decompress(b'x'), d2.unused_data

(b'', b'')

In [71]:
d2.decompress(data[24:32]), d2.unused_data

(b'Second', b'')

In [72]:
d2.decompress(data[32:]), d2.unused_data
# that unused_data is nonempty indicates the second stream of compression is complete and intact

(b' sentence', b'|')

## 5.8 Network exceptions
- the number of socket errors is quite large
- but the number of actual exceptions with socket operations is quite few
  - OSError: nearly raised by every failure at any stage in network transmission
  - socket.gaierror: raised when getaddrinfo() failed to find a name or service
  - socket.timeout: indicates timeout before operation could complete normally
  - herror: raised from certain old-fasioned address lookup calls

In [76]:
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    s.connect(('nonexistent.website.com', 80))
except socket.gaierror as e: # e.errno => -2; e.strerror => 'Name or service not known'
    raise

gaierror: [Errno -2] Name or service not known

- Some higher-level socket-based protocols such as *httplib* from PSL allow
  - expose raw socket errors
  - catch and turn raw socket errors into their own kind of errors

In [ ]:
import http.client
h = http.client.HTTPConnection('nonexistent.website.com')
h.request('GET', '/')

- But *urllib2* hides this same error and raises URLError to be clean and neutral

In [ ]:
import urllib.request
urllib.request.urlopen('http://nonexistent.website.com/')

## 5.9 Catching and reporting network exceptions
- Granular exception handler
  - wrap every network call with a try...except clause
  - suitable for short programs
- Blanket exception handler
  - wrap blocks of code with clear purposes